In [3]:
import numpy as np
import pandas as pd

In [4]:
###tna = pd.read_csv('TNA.csv',parse_dates=['caldt'],dtype={'crsp_fundno':np.str,'crsp_portno':np.str})

In [5]:
tna = pd.read_csv('TNA.csv',parse_dates=['caldt'])

In [6]:
tna.head()

,summary_period2,crsp_fundno,caldt,tna_latest,crsp_portno,crsp_obj_cd
0,Q,31097.0,2017-03-31,10222.8,1000045.0,EDYB
1,Q,31097.0,2017-06-30,10307.9,1000045.0,EDYB
2,Q,31097.0,2017-09-29,10276.6,1000045.0,EDYB
3,AQ,31097.0,2017-12-29,10382.8,1000045.0,EDYB
4,Q,31098.0,2017-03-31,130.6,1000045.0,EDYB


In [14]:
tna = tna.dropna(subset=['tna_latest','crsp_portno','crsp_fundno'])

In [16]:
tna['crsp_fundno']=tna['crsp_fundno'].astype(np.int32)
tna['crsp_portno']=tna['crsp_portno'].astype(np.int32)

In [17]:
tna.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 122786 entries, 0 to 126703
Data columns (total 6 columns):
summary_period2    122786 non-null object
crsp_fundno        122786 non-null int32
caldt              122786 non-null datetime64[ns]
tna_latest         122786 non-null float64
crsp_portno        122786 non-null int32
crsp_obj_cd        122654 non-null object
dtypes: datetime64[ns](1), float64(1), int32(2), object(2)
memory usage: 5.6+ MB


In [18]:
###fundno to portno - will not allow me to create a non index portno b/c many indexes

tna_portno = tna.groupby(['crsp_portno','caldt'],as_index=False).agg({'tna_latest':'sum'})
###tna_portno = tna_portno.reset_index(level=['crsp_portno','caldt'])
tna_portno.rename(columns={'tna_latest':'portno_tna'},inplace=True) #update tna_portno in place or replace the file

tna_portno.head()

,crsp_portno,caldt,portno_tna
0,1000045,2017-03-31,14505.8
1,1000045,2017-06-30,14601.5
2,1000045,2017-09-29,14764.7
3,1000045,2017-12-29,14934.4
4,1000052,2017-03-31,12073.3


In [19]:
###merge portfolio tna with fundno tna
#tna2 = pd.merge(tna,tna_portno,left_on=['crsp_portno','caldt'],right_on=['crsp_portno','caldt'],how='left')

tna2 = pd.merge(tna,tna_portno,on=['crsp_portno','caldt'],how='left')
tna2 = tna2[['crsp_fundno','caldt','tna_latest','crsp_portno','portno_tna']]

In [20]:
tna2.head()

,crsp_fundno,caldt,tna_latest,crsp_portno,portno_tna
0,31097,2017-03-31,10222.8,1000045,14505.8
1,31097,2017-06-30,10307.9,1000045,14601.5
2,31097,2017-09-29,10276.6,1000045,14764.7
3,31097,2017-12-29,10382.8,1000045,14934.4
4,31098,2017-03-31,130.6,1000045,14505.8


In [21]:
tna2['weights'] = tna2['tna_latest'] / tna2['portno_tna']

In [22]:
###why does this show me the data? 
tna2.sort_values(['crsp_portno','caldt','crsp_fundno']).head(10)

,crsp_fundno,caldt,tna_latest,crsp_portno,portno_tna,weights
0,31097,2017-03-31,10222.8,1000045,14505.8,0.704739
4,31098,2017-03-31,130.6,1000045,14505.8,0.009003
8,31099,2017-03-31,1675.2,1000045,14505.8,0.115485
12,31100,2017-03-31,224.2,1000045,14505.8,0.015456
16,31101,2017-03-31,1380.7,1000045,14505.8,0.095183
20,49718,2017-03-31,479.7,1000045,14505.8,0.033070
24,56238,2017-03-31,392.6,1000045,14505.8,0.027065
1,31097,2017-06-30,10307.9,1000045,14601.5,0.705948
5,31098,2017-06-30,107.0,1000045,14601.5,0.007328
9,31099,2017-06-30,1585.5,1000045,14601.5,0.108585


In [23]:
### na_values = 'xyz' for replacing invalid obs
ret = pd.read_csv('Returns.csv',parse_dates=['caldt'],na_values='R')

In [24]:
ret.head()

,caldt,crsp_fundno,mtna,mret,mnav
0,2017-01-31,105,26.9,0.028147,13.15
1,2017-02-28,105,27.3,0.026616,13.50
2,2017-03-31,105,27.6,0.017778,13.74
3,2017-04-28,105,28.2,0.024745,14.08
4,2017-05-31,105,28.6,0.020597,14.37


In [25]:
ret = ret[['caldt','crsp_fundno','mret']]
ret.dropna(subset=['mret'],inplace=True)
#ret = ret[ret['mret'] != 'R']
#ret['mret']=ret['mret'].astype(np.float64)

In [26]:
ret.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 426063 entries, 0 to 430180
Data columns (total 3 columns):
caldt          426063 non-null datetime64[ns]
crsp_fundno    426063 non-null int64
mret           426063 non-null float64
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 13.0 MB


In [27]:
ret.isnull().sum()

caldt          0
crsp_fundno    0
mret           0
dtype: int64

In [28]:
ret_tna = pd.merge(ret,tna2,left_on=['crsp_fundno','caldt'],right_on=['crsp_fundno','caldt'],how='inner')

In [32]:
ret_tna.isnull().sum()

caldt          0
crsp_fundno    0
mret           0
tna_latest     0
crsp_portno    0
portno_tna     0
weights        0
dtype: int64

In [31]:
ret_tna.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 122186 entries, 0 to 122185
Data columns (total 7 columns):
caldt          122186 non-null datetime64[ns]
crsp_fundno    122186 non-null int64
mret           122186 non-null float64
tna_latest     122186 non-null float64
crsp_portno    122186 non-null int32
portno_tna     122186 non-null float64
weights        122186 non-null float64
dtypes: datetime64[ns](1), float64(4), int32(1), int64(1)
memory usage: 7.0 MB


In [33]:
ret_tna.sort_values('weights')

,caldt,crsp_fundno,mret,tna_latest,crsp_portno,portno_tna,weights
71307,2017-12-29,51681,-0.000011,-99.0,1030834,1.2,-82.500000
49528,2017-03-31,38480,0.003460,-99.0,1023801,14.8,-6.689189
71305,2017-06-30,51681,-0.000062,-99.0,1030834,16.7,-5.928144
117805,2017-03-31,90379,-0.002881,-99.0,1026971,19.1,-5.183246
71306,2017-09-29,51681,-0.002090,-99.0,1030834,21.8,-4.541284
49529,2017-06-30,38480,0.005300,-99.0,1023801,22.1,-4.479638
17360,2017-03-31,14668,0.000000,77.3,1023384,-21.7,-3.562212
49530,2017-09-29,38480,0.000000,-99.0,1023801,29.3,-3.378840
17361,2017-06-30,14668,-0.000959,74.9,1023384,-24.1,-3.107884
17362,2017-09-29,14668,-0.005820,73.3,1023384,-25.7,-2.852140


In [34]:
ret_tna['crsp_portno']=ret_tna['crsp_portno'].astype(np.int32)

In [35]:
ret_tna.head()

,caldt,crsp_fundno,mret,tna_latest,crsp_portno,portno_tna,weights
0,2017-03-31,105,0.017778,27.6,1022431,63.3,0.436019
1,2017-06-30,105,0.004871,28.0,1022431,62.8,0.445860
2,2017-09-29,105,0.027609,29.0,1022431,64.2,0.451713
3,2017-12-29,105,0.010415,29.6,1022431,65.1,0.454685
4,2017-03-31,2704,0.002708,405.4,1026386,491.4,0.824990


In [36]:
ret_tna['weight_ret'] = ret_tna['weights'] * ret_tna['mret']

In [37]:
ret_tna.head()

,caldt,crsp_fundno,mret,tna_latest,crsp_portno,portno_tna,weights,weight_ret
0,2017-03-31,105,0.017778,27.6,1022431,63.3,0.436019,0.007752
1,2017-06-30,105,0.004871,28.0,1022431,62.8,0.445860,0.002172
2,2017-09-29,105,0.027609,29.0,1022431,64.2,0.451713,0.012471
3,2017-12-29,105,0.010415,29.6,1022431,65.1,0.454685,0.004736
4,2017-03-31,2704,0.002708,405.4,1026386,491.4,0.824990,0.002234


In [38]:
ret_tna2 = ret_tna.groupby(['crsp_portno','caldt'],as_index=False).agg({'weight_ret':'sum'})
#ret_tna2 = ret_tna2.reset_index(level=['crsp_portno','caldt'])
ret_tna2.head()

,crsp_portno,caldt,weight_ret
0,1000045,2017-03-31,-0.007437
1,1000045,2017-06-30,0.016478
2,1000045,2017-09-29,0.027253
3,1000045,2017-12-29,0.010968
4,1000052,2017-03-31,-0.014970


In [39]:
###LAGGING RETURNS BY 1 PERIOD

ret_tna2['lag_ret'] = ret_tna2.groupby('crsp_portno')['weight_ret'].shift(1)
ret_tna2.head(20)

,crsp_portno,caldt,weight_ret,lag_ret
0,1000045,2017-03-31,-0.007437,NaN
1,1000045,2017-06-30,0.016478,-0.007437
2,1000045,2017-09-29,0.027253,0.016478
3,1000045,2017-12-29,0.010968,0.027253
4,1000052,2017-03-31,-0.014970,NaN
5,1000052,2017-06-30,0.022952,-0.014970
6,1000052,2017-09-29,0.051748,0.022952
7,1000052,2017-12-29,0.023599,0.051748
8,1000103,2017-03-31,-0.011426,NaN
9,1000103,2017-06-30,0.020814,-0.011426
